# Домашнее задание № 7

## по дисциплине: Теория вероятностей и математическая статистика

## Тема: Линейная регрессия. Двухвыборочный t-тест. A/B-тестирование

### выполнил: Евгений Кириллов

__Задача 1__

Дана матрица объект-признак
```
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]
```
и значения целевой переменной
```
y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]
```

Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат. В качестве ответа запишите значение коэффициента детерминации для модели на этих признаках.

__Ответ__: 0.76342462

In [1]:
import numpy as np
from itertools import combinations
from scipy import stats

In [2]:
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]

y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]

In [3]:
X = np.array(X)
y = np.array(y)

для решения задачи нужно найти коэффициент детерминации, напишем для этого функцию:

In [4]:
def determination_coefficient(x, y):
    # функция считает коэффициент детерминации
    
    def sum_of_squares(samples):
        # функция считает сумму квадратов отклонений.

        return ((samples - samples.mean()) ** 2).sum()
    
    x = np.hstack([np.ones((x.shape[0], 1)), x])
    xtx = x.T.dot(x)
    xtx_inv = np.linalg.inv(xtx)
    b = xtx_inv.dot(x.T).dot(y)
    z = x.dot(b)
    E = y - z
    
    return 1 - sum_of_squares(E) / sum_of_squares(y)

посчитаем для каждой пары признаков коэффициент детерминации:

In [5]:
for feature_1, feature_2 in combinations(range(X.shape[1]), 2):
    r = determination_coefficient(X[:, [feature_1, feature_2]], y)
    print(f'{feature_1}-{feature_2}: {r}')

0-1: 0.18113594742585204
0-2: 0.7634246238793152
0-3: 0.45329667831440745
1-2: 0.5479482734039012
1-3: 0.6062055761129932
2-3: 0.6224419876505322


использование первого и третьего признаков (0-2) даёт наибольшее значение - 0.7634246238793152

__Задача 2__

Является ли значимым уравнение регрессии, полученное в предыдущей задаче? В качестве ответа запишите 1, если является, и 0 иначе.

__Ответ__: 1

для проверки статистической зависимости нам понадобится распределение Фишера:

In [6]:
k1 = 2
k2 = X.shape[0] - k1 - 1
R = determination_coefficient(X[:, [0, 2]], y)

F = (R / k1) / ((1 - R) / k2)
F

11.29443912292265

найдём границу критической области:

In [7]:
alpha = 0.05
stats.f.ppf(1 - alpha, k1, k2)

4.73741412777588

4.73741412777588 < 11.29443912292265 - значение статистики попало в критическую область, а значит, уравнение регрессии является статистически значимым

__Задача 3__

Для проведения A/B-тестирования сайта интернет-магазина были получены следующие данные: страница `A` была посещена 2509 раз, из них 77 закончились совершением покупки, страница `B` была посещена 1465 раз, 60 из них закончились совершением покупки. Является ли значимым отличие конверсии на страницах `A` и `B`? В качестве ответа запишите 1, если является, и 0 иначе.

_Подсказка_. Реализуйте двухвыборочный t-тест. В качестве выборок здесь можно взять наборы меток совершения покупки (0 или 1) каждым посетителем.

__Ответ__: 0

In [8]:
x1 = np.zeros(2509)
x1[np.arange(77)] = 1

x2 = np.zeros(1465)
x2[np.arange(60)] = 1

посчитаем $\sigma_\Delta$:

In [9]:
n1 = x1.size
n2 = x2.size

s1 = x1.std(ddof=1)
s2 = x2.std(ddof=1)

s_delta = np.sqrt(s1 ** 2 / n1 + s2 ** 2 / n2)
s_delta

0.006220171278295827

значение статистики:

In [10]:
t = (x1.mean() - x2.mean()) / s_delta
t

-1.6504551408398205

посчитаем число степеней свободы:

In [11]:
df = (s1 ** 2 / n1 + s2 ** 2 / n2) ** 2 / ((s1 ** 2 / n1) ** 2 / (n1 - 1) + (s2 ** 2 / n2) ** 2 / (n2 - 1))
df

2732.8025644352133

посчитаем квантили распределения Стьюдента:

In [12]:
alpha = 0.05
stats.t.ppf(alpha / 2, df=df), stats.t.ppf(1 - alpha / 2, df=df)

(-1.9608324352746576, 1.9608324352746571)

-1.9608324352746576 < -1.6504551408398205 < 1.9608324352746571 - значение статистики в критическую область не попало, следовательно, значимого отличия между конверсией на страницах A и B нет